# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [25]:
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import normal, uniform, choice
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [26]:
ws = Workspace.from_config()
experiment_name = 'capstone-hyperparameter-experiment'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: UMRAIMLND
Azure region: centralus
Subscription id: b1d4fcba-3863-40a9-84b1-4b0f82fd16cc
Resource group: rg-UMRDataExternalization-dev


In [27]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [28]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "HeartDisease"
description_text = "Heart data"
ws = Workspace.from_config()

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
       # example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        example_data = 'https://raw.githubusercontent.com/sireeshag09/udacity-capstone/main/heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
df = dataset.to_pandas_dataframe()
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [29]:
dataset.take(5).to_pandas_dataframe()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,False,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,False,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,False,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,True,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,False,0.0,Up,0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [35]:
from azureml.core import Environment, ScriptRunConfig, Workspace

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=3)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({'--batch_size' : choice(10,20,30,40),'--frequency': choice(200,400,600)})

#TODO: Create your estimator and hyperdrive config
src = ScriptRunConfig(source_directory= '.',
                      command=['python','train.py'],
                      compute_target=compute_target,
                      environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling= param_sampling,
                                    primary_metric_name="AUC_weighted",
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=8, 
                                    max_concurrent_runs=4,
                                    policy = early_termination_policy,
                                    run_config=src)

In [36]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [37]:
RunDetails(hyperdrive_run).show
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_d37f9add-2aed-4f54-88e1-439e00f29682
Web View: https://ml.azure.com/runs/HD_d37f9add-2aed-4f54-88e1-439e00f29682?wsid=/subscriptions/b1d4fcba-3863-40a9-84b1-4b0f82fd16cc/resourcegroups/rg-UMRDataExternalization-dev/workspaces/UMRAIMLND&tid=db05faca-c82a-4b9d-b9c5-0f64b6755421

Streaming azureml-logs/hyperdrive.txt

[2023-02-05T06:42:36.025622][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-02-05T06:42:36.6494352Z][SCHEDULER][INFO]Scheduling job, id='HD_d37f9add-2aed-4f54-88e1-439e00f29682_0' 
[2023-02-05T06:42:36.7214688Z][SCHEDULER][INFO]Scheduling job, id='HD_d37f9add-2aed-4f54-88e1-439e00f29682_1' 
[2023-02-05T06:42:36.7807378Z][SCHEDULER][INFO]Scheduling job, id='HD_d37f9add-2aed-4f54-88e1-439e00f29682_2' 
[2023-02-05T06:42:36.8968973Z][SCHEDULER][INFO]Scheduling job, id='HD_d37f9add-2aed-4f54-88e1-439e00f29682_3' 
[2023-02-05T06:42:36.856637][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.

{'runId': 'HD_d37f9add-2aed-4f54-88e1-439e00f29682',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2023-02-05T06:42:35.389276Z',
 'endTimeUtc': '2023-02-05T06:46:36.602888Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"AUC_weighted","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '63e2d8f5-c149-4790-abea-d2f538fc6a3c',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '12',
  'score': '0.9415233415233415',
  'best_child_run_id': 'HD_d37f9add-2aed-4f54-88e1-439e00f29682_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_d37f9add-2aed-4f54-88e1-439e00f29682_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientT

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [38]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
 
print('Best Run Id: ', best_run.id)
print('AUC_weighted of Best Run is:', best_run_metrics['AUC_weighted'])
print('Parameter Values are:',best_run.get_details()['runDefinition']['arguments'])

Best Run Id:  HD_d37f9add-2aed-4f54-88e1-439e00f29682_0
AUC_weighted of Best Run is: 0.9415233415233415
Parameter Values are: []


In [39]:
import joblib
# Get your best run and save the model from that run.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("Best run metrics :",best_run.get_metrics())

print("Best run details :",best_run.get_details())

print("Best run file names :",best_run.get_file_names())

[{'run_id': 'HD_d37f9add-2aed-4f54-88e1-439e00f29682_7', 'hyperparameters': None, 'best_primary_metric': 0.9415233415233415, 'status': 'Completed'}, {'run_id': 'HD_d37f9add-2aed-4f54-88e1-439e00f29682_6', 'hyperparameters': None, 'best_primary_metric': 0.9415233415233415, 'status': 'Completed'}, {'run_id': 'HD_d37f9add-2aed-4f54-88e1-439e00f29682_5', 'hyperparameters': None, 'best_primary_metric': 0.9415233415233415, 'status': 'Completed'}, {'run_id': 'HD_d37f9add-2aed-4f54-88e1-439e00f29682_4', 'hyperparameters': None, 'best_primary_metric': 0.9415233415233415, 'status': 'Completed'}, {'run_id': 'HD_d37f9add-2aed-4f54-88e1-439e00f29682_3', 'hyperparameters': None, 'best_primary_metric': 0.9415233415233415, 'status': 'Completed'}, {'run_id': 'HD_d37f9add-2aed-4f54-88e1-439e00f29682_2', 'hyperparameters': None, 'best_primary_metric': 0.9415233415233415, 'status': 'Completed'}, {'run_id': 'HD_d37f9add-2aed-4f54-88e1-439e00f29682_0', 'hyperparameters': None, 'best_primary_metric': 0.94152

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

